# ETL 04 - Scripts Area Stage SQL
    Objetivo: Popular as tabelas no banco DDTM_OiMasterDados
    Desenvolvedor: Marcelo Muniz de Alencar

In [ ]:
# importar pacote pandas
import pandas as pd 

# importar sqlalchemy, urllib, pyodbc
from sqlalchemy import create_engine
import urllib
import pyodbc

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# conexão banco de dados 'DDTM_OiMasterDados' no servidor
connDDTM = pyodbc.connect('Driver={SQL Server};'
                      'Server=PCMARCELO\SQLEXPRESS;' # alterar linha para seu servidor
                      'Database=DDTM_OiMasterDados;'
                      'Trusted_Connection=yes;',autocommit=True)

In [ ]:
# populando a tabela TOiDIM_Calendario
cursorDDTM = connDDTM.cursor()

sqlDDTM_OiMasterDados_Calendario = ''' 
DELETE FROM DDTM_OiMasterDados.OiMasterDados.TOiDIM_Calendario

INSERT INTO DDTM_OiMasterDados.OiMasterDados.TOiDIM_Calendario
SELECT DISTINCT 
	   T1.[NumData] as CodSkData
	  ,convert(date,T1.[DescData]) as CodBkData
      ,T1.[NumAno]
      ,T1.[DescAno]
      ,T1.[NumSemestre]
      ,T1.[DescSemestre]
      ,T1.[NumTrimestre]
      ,T1.[DescTrimestre]
      ,T1.[NumMes]
      ,T1.[DescMes]
      ,T1.[NumSemanaAno]
      ,T1.[DescSemanaAno]
      ,T1.[NumDia]
      ,T1.[NumDiaSemana]
      ,T1.[DescDiaSemana]
      ,T1.[FlgFinalSemana]
	 ,CASE 
			WHEN (NumMes = 6 and NumDia >= 21)
				OR (NumMes = 7)
				OR (NumMes = 8)
				OR (NumMes = 9 and NumDia <=22)
			THEN 'Verão'
			WHEN (NumMes = 9 and NumDia >= 23)
				OR (NumMes = 10)
				OR (NumMes = 11)
				OR (NumMes = 12 and NumDia <=20)
			THEN 'Outono'
			WHEN (NumMes = 12 and NumDia >= 21)
				OR (NumMes = 1)
				OR (NumMes = 2)
				OR (NumMes = 3 and NumDia <=19)
			THEN 'Inverno'
			else 'Primavera'
		end as DescEstacaoBoreal
		,CASE WHEN T1.[NumData] = 19000101
				THEN '*** Estação Não Informada'
			WHEN (NumMes = 6 and NumDia >= 21)
				OR (NumMes = 7)
				OR (NumMes = 8)
				OR (NumMes = 9 and NumDia <=22)
			THEN 'Inverno'
			WHEN (NumMes = 9 and NumDia >= 23)
				OR (NumMes = 10)
				OR (NumMes = 11)
				OR (NumMes = 12 and NumDia <=20)
			THEN 'Primavera'
			WHEN (NumMes = 12 and NumDia >= 21)
				OR (NumMes = 1)
				OR (NumMes = 2)
				OR (NumMes = 3 and NumDia <=19)
			THEN 'Verão'
			else 'Outono'
		end as DescEstacaoAustral
FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_Calendario] AS T1 (NOLOCK)
		INNER JOIN
	 [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_FatoTemperatura] AS T2 (NOLOCK)
		ON T1.NumData = T2.NumData
ORDER BY 1
'''

try:
    cursorDDTM.execute(sqlDDTM_OiMasterDados_Calendario)
    connDDTM.commit()
    print('Executado o script SQL para criação da tabela TOiDIM_Calendario.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para criação da tabela T999DIM_Calendario.


In [ ]:
# populando a tabela TOiDIM_Cidade
cursorDDTM = connDDTM.cursor()

sqlDDTM_OiMasterDados_Cidade = ''' 
DELETE FROM DDTM_OiMasterDados.OiMasterDados.TOiDIM_Cidade

INSERT INTO DDTM_OiMasterDados.OiMasterDados.TOiDIM_Cidade
select	-1 as CodSkCidade, 
		'*** Cidade Desconhecida'  as NomCidade,
		'*** Estado Desconhecido'  as NomEstado,
		'*** País Desconhecido'	   as NomPais,
		'*** Região Gloco Desconhecida'  
								   as NomRegiaoGlobo,
		0					   as ValLatitude, 
		0					   as ValLongitude,
		'X' 				       as FlgHemisferioNorte

union all

select	t1.NumCidade as CodSkCidade, 
		t1.Cidade	as NomCidade,
		case when t1.Estado IS NULL 
				then '*** Estado Não Informado'
				else t1.Estado
		end as NomEstado,
		t1.Pais as NomPais,
		t1.RegiaoGlobo as NomRegiaoGlobo,
		t2.ValLatitude, 
		t2.ValLongitude,
		CASE WHEN t2.ValLatitude > 0
				THEN 'S'
				ELSE 'N'
		END as FlgHemisferioNorte
from
(SELECT [NumCidade]
      ,case when [Cidade] = 'Montvideo'
				then 'Montevideo'
				else [Cidade]
	    end as [Cidade]
      ,[Estado]
      ,[Pais]
      ,[RegiaoGlobo]
	  ,case when rtrim(ltrim(Estado)) <> ''
				then Cidade + ', ' + Estado + ', '+ Pais
                else Cidade + ', ' + Pais
       end as Localizacao
FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_Cidade] (NOLOCK)) as t1
left outer join
[DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_CidadeGeoLocalizacao] as t2 (NOLOCK)
on t1.Localizacao = t2.Localizacao
or t1.Cidade = t2.Localizacao
order by 1
'''

try:
    cursorDDTM.execute(sqlDDTM_OiMasterDados_Cidade)
    connDDTM.commit()
    print('Executado o script SQL para criação da tabela TOiDIM_Cidade.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para criação da tabela T999DIM_Cidade.


In [ ]:
# populando a tabela TOiFAT_Temperatura
cursorDDTM = connDDTM.cursor()

sqlDDTM_OiMasterDados_Temperatura = ''' 
DELETE FROM [DDTM_OiMasterDados].[OiMasterDados].[TOiFAT_Temperatura] 

INSERT into [DDTM_OiMasterDados].[OiMasterDados].[TOiFAT_Temperatura] 
SELECT [NumData]				as CodSkData
      ,t1.[NumCidade]			as CodSkLocalizacao
      ,[ValMedioTemperaturaC]
FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_FatoTemperatura] AS T1 (NOLOCK)
ORDER BY 1,2
'''

try:
    cursorDDTM.execute(sqlDDTM_OiMasterDados_Temperatura)
    connDDTM.commit()
    print('Executado o script SQL para criação da tabela TOiFAT_Temperatura.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para criação da tabela T999DIM_Cidade.


## FIM